Import arcpy, ustawienie środowiska projektu

In [18]:
import arcpy
from arcpy import *

arcpy.env.workspace = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdp")
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
arcpy.env.overwriteOutput = True
arcpy.env.extent = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer")
arcpy.env.mask = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer.shp")
arcpy.env.cellSize = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\raster_extracted.tif")

aprx = arcpy.mp.ArcGISProject("CURRENT")  # Otwiera bieżący projekt
m = aprx.activeMap

# wylaczenie dodawania warstw
arcpy.env.addOutputsToMap = False

Wczytanie warstw i ich zapis do zmiennych

In [2]:
drogi_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\SKDR.shp")
rzeki = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\SWRS.shp")
jeziora = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\PTWP.shp")
lasy = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\PTLZ.shp")
budynki_wszystkie = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\BUBDA.shp")
bufor = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\swieradow_zdroj_Buffer")
raster_gmina = (R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\raster_extracted")

Selekcja po atrybutach odpowiednich warstw

In [3]:
#wyselekcjonowanie drog utwardzonych
drogi = arcpy.management.SelectLayerByAttribute(drogi_wszystkie, "NEW_SELECTION", "MATE_NAWIE IN ('beton','bruk', 'kostka kamienna', 'kostka prefabrykowana', 'masa bitumiczna', 'płyty betonowe')", None)
arcpy.management.CopyFeatures(drogi, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\drogi_selected", '', None, None, None)
drogi_selected = 'drogi_selected'
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\drogi_selected")
arcpy.SelectLayerByAttribute_management(drogi, "CLEAR_SELECTION")

#wyselekcjonowanie budynkow mieszkalnych
budynki = arcpy.management.SelectLayerByAttribute(budynki_wszystkie, "NEW_SELECTION", "FOBUD IN ('budynki mieszkalne')", None)
arcpy.management.CopyFeatures(budynki, r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\budynki_selected", '', None, None, None)
budynki_selected = 'budynki_selected'
arcpy.SelectLayerByAttribute_management(budynki, "CLEAR_SELECTION")

<Result 'BUBDA_Layer1'>

Stworzenie map odległości dla kryteriów narzędziem Euclidian Distance

In [4]:
distance_rzeki = arcpy.sa.EucDistance(rzeki, None, "5")
distance_jeziora = arcpy.sa.EucDistance(jeziora, None, "5")
distance_drogi = arcpy.sa.EucDistance(drogi_selected, None, "5")
distance_budynki = arcpy.sa.EucDistance(budynki_selected, None, "5")
distance_lasy = arcpy.sa.EucDistance(lasy, None, "5")

# mapa odleglosci do kryterium z wezlami wykonana w QGiS:
distance_wezly  = Raster(r"C:\Users\mplaciszewska\Desktop\studia\geoinf_sem_5\analizy_przestrzenne\cw1\kryterium_wezly_komunikacyjne.tif")

Kryterium 1 - odległość od rzek i zbiorników wodnych, jak najbliżej (nieprzekraczalna strefa ochronna 100m)

In [5]:
#kryterium ostre dla strefy ochronnej 100m poprzez reklasyfikacje map odleglosci
rzeki_ostre = arcpy.ddd.Reclassify(distance_rzeki, "VALUE", f"0 100 0;100 {distance_rzeki.maximum} 1")
jeziora_ostre = arcpy.ddd.Reclassify(distance_jeziora, "VALUE", f"0 100 0;100 {distance_jeziora.maximum} 1")

#połączenie kryteriów ostrych dla rzek i jezior
wody_ostre = arcpy.sa.FuzzyOverlay([rzeki_ostre, jeziora_ostre], "AND")

In [6]:
#kryterium rozmyte dla obszarów w odległości powyżej 100m
rzeki_rozmyte = arcpy.sa.FuzzyMembership(distance_rzeki, arcpy.sa.FuzzyLinear(400, 200))
jeziora_rozmyte = arcpy.sa.FuzzyMembership(distance_jeziora, arcpy.sa.FuzzyLinear(400, 200))

#połączenie kryteriów rozmytych dla rzek i jezior
wody_rozmyte = arcpy.sa.FuzzyOverlay([rzeki_rozmyte, jeziora_rozmyte], "OR")

In [7]:
#połączenie kryterium ostrego i rozmytego dla wody
kryterium_wody = arcpy.sa.FuzzyOverlay([wody_ostre, wody_rozmyte], 'AND')
kryterium_wody.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wody")

Kryterium 2 - dostęp do dróg utwardzonych, jak największe zagęszczenie 

In [44]:
# ocena przydatnosci pod wzgledem zageszczenia
raster_zageszczenia = arcpy.sa.LineDensity(
    in_polyline_features=drogi_selected,
    population_field="NONE",
    cell_size=5,
    search_radius=1000,
    area_unit_scale_factor="SQUARE_METERS"
)
arcpy.management.CalculateStatistics(raster_zageszczenia)

max_value = raster_zageszczenia.maximum

# przeskalowanie rastra zageszczenia do wartosci 0-1
kryterium_drogi = arcpy.sa.RescaleByFunction(
    in_raster=raster_zageszczenia,
    transformation_function= f"LINEAR 0 1,{max_value} 0 # {max_value} #",
    from_scale=0,
    to_scale=1
)

kryterium_drogi.save(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_drogi")
# m.addDataFromPath(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_drogi")

Kryterium 3 - odległość od budynków mieszkalnych

In [9]:
kryterium_budynki = arcpy.sa.FuzzyMembership(distance_budynki,  arcpy.sa.FuzzyLinear(150, 400))
kryterium_budynki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_budynki")

Krterium 4 - pokrycie terenu, nie w lesie, powyżej 15m od lasu, optymalnie 100m od lasu 

In [10]:
kryterium_lasy = arcpy.sa.FuzzyMembership(distance_lasy, arcpy.sa.FuzzyLinear(15, 100))
kryterium_lasy.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_lasy")

Kryterium 5 - nachylenie stoków, optymalnie płasko, maksymalnie 10%

In [11]:
#stworzenie warstwy z nachyleniami stoków narzędziem Slope
spadki_output = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\spadki"
spadki = arcpy.ddd.Slope(raster_gmina, spadki_output, "PERCENT_RISE", 1)

kryterium_spadki = arcpy.sa.FuzzyMembership(spadki, arcpy.sa.FuzzyLinear(10, 3))
kryterium_spadki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_spadki")

Kryterium 6 - nasłonecznienie, optymalnie stoki południowe (SW-SE)

In [12]:
#stworzenie warstwy ze stokami narzędziem Aspect
stoki_output = R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\stoki"
stoki = arcpy.ddd.Aspect(raster_gmina, stoki_output)

# rozdzielenie kryterium rozmytego na rosnący i malejący
stoki_ES = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(90, 113))
stoki_WS = arcpy.sa.FuzzyMembership(stoki, arcpy.sa.FuzzyLinear(270, 248))

# ustawienie przydatności płaskich stoków na 1
plaskie_stoki = arcpy.sa.Con(stoki, 1, 0, "VALUE = -1") 

stoki_rozmyte = arcpy.sa.FuzzyOverlay([stoki_ES, stoki_WS], 'AND')
kryterium_stoki = arcpy.sa.FuzzyOverlay([stoki_rozmyte, plaskie_stoki], 'OR')

kryterium_stoki.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_stoki")

Utworzenie węzłów komunikacyjnych

In [13]:
kryterium_wezly = arcpy.sa.FuzzyMembership(distance_wezly, arcpy.sa.FuzzyLinear(distance_wezly.maximum, 200))
kryterium_wezly.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wezly")
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\kryterium_wezly")

Połączenie kryteriów 

In [48]:
# połączenie kryteriów stosując równe wagi dla kryteriów
kryteria_rowne = arcpy.sa.WSTable(
    [
        [kryterium_budynki, "VALUE", 0.14285714],
        [kryterium_drogi, "VALUE", 0.14285714],
        [kryterium_lasy, "VALUE", 0.14285714],
        [kryterium_spadki, "VALUE", 0.14285714],
        [kryterium_stoki, "VALUE", 0.14285714],
        [kryterium_wody, "VALUE", 0.14285714],
        [kryterium_wezly, "VALUE", 0.14285714],
    ]
)
polaczone_kryteria_rowne = arcpy.sa.WeightedSum(kryteria_rowne)
polaczone_kryteria_rowne.save(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\polaczone_kryteria_rowne")

# połączenie kryteriów stosując różne wagi dla kryteriów
kryteria_rozne = arcpy.sa.WSTable(
    [
        [kryterium_budynki, "VALUE", 0.14],
        [kryterium_drogi, "VALUE", 0.1],
        [kryterium_lasy, "VALUE", 0.125],
        [kryterium_spadki, "VALUE", 0.18],
        [kryterium_stoki, "VALUE", 0.18],
        [kryterium_wody, "VALUE", 0.18],
        [kryterium_wezly, "VALUE", 0.095],
    ]
)
polaczone_kryteria_rozne = arcpy.sa.WeightedSum(kryteria_rozne)
polaczone_kryteria_rozne.save(r"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\polaczone_kryteria_rozne")
print("ok")


ok


Uwzględnienie kryteriów ostrych do map przydatności (nieprzekraczalne strefy w kryterium wody, budynków i lasów)

In [59]:
# budynki ostre:
budynki_ostre = arcpy.ddd.Reclassify(distance_budynki, "VALUE", f"0 150 0; 150 {distance_budynki.maximum} 1")

# lasy ostre:
lasy_ostre = arcpy.ddd.Reclassify(distance_lasy, "VALUE", f"0 15 0; 15 {distance_lasy.maximum} 1")

def times(in_raster1, in_raster2):
    out_raster = arcpy.ia.Times(
        in_raster_or_constant1=in_raster1,
        in_raster_or_constant2=in_raster2
    )
    return out_raster

# przemnożenie mapy przydatności o równych wagach przez kryteria ostre:
def polacz_ostre(polaczone_kryteria):
    mapa_przydatnosci1 = times(polaczone_kryteria, wody_ostre)
    mapa_przydatnosci2 = times(mapa_przydatnosci1, budynki_ostre)
    mapa_przydatnosci = times(mapa_przydatnosci2, lasy_ostre)

    return mapa_przydatnosci

mapa_przydatnosci_rowne = polacz_ostre(polaczone_kryteria_rowne)
mapa_przydatnosci_rozne = polacz_ostre(polaczone_kryteria_rozne)

mapa_przydatnosci_rowne.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_rowne")
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_rowne")
mapa_przydatnosci_rozne.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_rozne")
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_rozne")

reklasyfikacja map przydatnosci w celu wyodrębnienia obszarów o 80% przydatności

In [62]:
def reklasyfikuj_mape(mapa_przydatnosci):
    granica = 80/100 * mapa_przydatnosci.maximum
    mapa_reclass = arcpy.sa.Reclassify(mapa_przydatnosci, "VALUE", f"0 {granica} 0; {granica} {mapa_przydatnosci.maximum} 1")
    return mapa_reclass

mapa_przydatnosci_reclass_rowne = reklasyfikuj_mape(mapa_przydatnosci_rowne)
mapa_przydatnosci_reclass_rozne = reklasyfikuj_mape(mapa_przydatnosci_rozne)
mapa_przydatnosci_reclass_rowne.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_reclass_rowne")
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_reclass_rowne")
mapa_przydatnosci_reclass_rozne.save(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_reclass_rozne")
m.addDataFromPath(R"C:\Users\mplaciszewska\Documents\ArcGIS\Projects\projekt1-analizyp\projekt1-analizyp.gdb\mapa_przydatnosci_reclass_rozne")

Wczytanie działek